### Import data

In [1]:
include("importData.jl")
include("Hill.jl")
include("plot.jl")

# import G1, G2, and population data
conc_t, popt, g2t, g1t, g2_0t, g1_0t = setup_data("paclitaxel");

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:29
└ @ Main /home/farnazm/dde/lapatinib/DrugResponseModel.jl/src/importData.jl:29
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:37
└ @ Main /home/farnazm/dde/lapatinib/DrugResponseModel.jl/src/importData.jl:37


### Hill model, residual functions and optimization

In [ ]:
#  [EC50, b_steepness, alpha_min, alpha_max, beta_min, beta_max, tau1_mean, tau1_max, tau2_min, tau2_max, gamma1_max, gamma2_max]
# guess
guess = [[1.56, 0.05, 0.007, 0.008, 0.004, 0.05, 34.286, 6.56, 8.0, 7.0, 0.005, 0.035], 
[1.8, 0.02, 0.008, 0.008, 0.03, 0.0045, 20.0, 6.0, 20.0, 10.0, 0.003, 0.02],
[1.9, 0.03, 0.006, 0.007, 0.05, 0.0035, 15.0, 5.0, 30.0, 5.0, 0.002, 0.03],
[2.1, 0.04, 0.007, 0.005, 0.03, 0.005, 30.0, 3.0, 20.0, 4.0, 0.003, 0.02],
[1.8664, 0.097836, 0.00682578, 0.00958013, 0.00503788, 0.0433964, 38.941, 5.49712, 9.94827, 7.20807, 0.00294624, 0.0222821]]
# max num of steps
num_steps=1e3
best_fit = zeros(5)
params = zeros(12, 5)
for i in 1:5 
    best_fit[i], params[:, i] = optimize_hill(guess[i], conc_t, g1t, g2t, g1_0t, g2_0t, num_steps)
end


global optimization begins ...
Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64},RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
50.03 secs, 393 evals, 302 steps, improv/step: 0.470 (last = 0.4702), fitness=502007.593673392
100.05 secs, 780 evals, 684 steps, improv/step: 0.402 (last = 0.3482), fitness=498544.517755446

Optimization stopped after 1001 steps and 140.86 seconds
Termination reason: Max number of steps (1000) reached
Steps per second = 7.11
Function evals per second = 7.79
Improvements/step = 0.37200
Total function evaluations = 1097


Best candidate found: [2.25536, 16.8012, 0.00746059, 0.00904615, 0.00154485, 0.0369074, 21.5996, 4.64416, 9.13281, 4.60996, 0.00516123, 0.027216]

Fitness: 496659.445930775

the fitness before applying local optimization 496659.4459307755
global optimization begins ...
Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64},Ra

In [ ]:
best, arg = findmin(best_fit)
ps = params[:,arg]

### Plot the data with the new set of parameters for DDE

In [ ]:
# easy way to convert the hill parameters to DDE parameters
p = getDDEparams(ps, conc_t)
# i showas the trial number, which could be from 1:control, ..., 8: maximum drug concentraation
p_1 = plotIt(p[:, 1], g1t, g2t, g2_0t, g1_0t, popt, 1, "Hill paclitaxel")
p_2 = plotIt(p[:, 2], g1t, g2t, g2_0t, g1_0t, popt, 2, "")
p_3 = plotIt(p[:, 3], g1t, g2t, g2_0t, g1_0t, popt, 3, "")
p_4 = plotIt(p[:, 4], g1t, g2t, g2_0t, g1_0t, popt, 4, "")
p_5 = plotIt(p[:, 5], g1t, g2t, g2_0t, g1_0t, popt, 5, "")
p_6 = plotIt(p[:, 6], g1t, g2t, g2_0t, g1_0t, popt, 6, "")
p_7 = plotIt(p[:, 7], g1t, g2t, g2_0t, g1_0t, popt, 7, "")
p_8 = plotIt(p[:, 8], g1t, g2t, g2_0t, g1_0t, popt, 8, "")
plot(p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8)
plot!(size = (1000, 1000))

### Plot Hill curve for all of the DDE model parameters

In [ ]:
# plot
p__1 = plot(conc_t, p[1, :], label = "alpha", linewidth = 2, xlabel = "concentration [nM]", ylabel = "drug effect", line = (0.5, 4, :red))
p__2 = plot(conc_t, p[2, :], label = "beta", linewidth = 2, legend =:best, line = (0.5, 4, :red))
p__3 = plot(conc_t, p[3, :], label = "tau1", linewidth = 2, line = (0.5, 4, :red))
p__4 = plot(conc_t, p[4, :], label = "tau2", linewidth = 2, line = (0.5, 4, :red))
p__5 = plot(conc_t, p[5, :], label = "gamma1", linewidth = 2, line = (0.5, 4, :red))
p__6 = plot(conc_t, p[6, :], label = "gamma2", linewidth = 2, line = (0.5, 4, :red))
plot(p__1, p__2, p__3, p__4, p__5, p__6)
plot!(size = (1000, 700))